In [4]:


fastRecordIconPositionsX = [1136, 1171, 1207, 1244, 1280, 1316, 1352, 1388, 1424, 
1424, 
1424, 1424, 1352, 1316, 1280, 1244, 1207, 1171, 1136, 
1136, 1171, 1207, 1244, 1280, 1316, 1352, 1388]

fastRecordIconPositionsY = [611, 611, 611, 611, 611, 611, 611, 611, 611, 
637, 
677, 677, 677, 677, 677, 677, 677, 677, 677,
637, 637, 637, 637, 637, 637, 637, 637]

assert len(fastRecordIconPositionsX) == len(fastRecordIconPositionsY)

In [5]:
import pyautogui
import os
from tqdm import tqdm, trange

pyautogui.FAILSAFE = True

# check OS 
userOS = os.name

if "nt" in userOS: 
    directory = os.getcwd() + "\\in-new"
elif "posix" in userOS:
    directory = os.getcwd() + "/in-new"
else: 
    directory = None

file_names = []

for file in os.listdir(directory):
    if file.endswith(".txt") and "log" not in file:
        file_names.append(file[:-4])

# List of players, source from directory with previously scraped player names 
playerList = sorted(file_names)
renamed = ["BacardiSprite", "BearchenGott", "Livgo", "Lubotter2009", 'JustK1lling', 'schmaarek', "ymq_", "zTotito", "Chrisi20101", "Kokochampi"]

for i in renamed: 
    try: playerList.remove(i)
    except: ValueError(i + " not in list.")

# add new players: 
newPlayers = ["Vacted", "camman18YT", "lvlaurin"]

for i in newPlayers:
    if i not in playerList:
        playerList.append(i)

playerList = sorted(playerList)

# Check for .txt files in the desktop path and add them to a list, then remove .txt from each string and remove all matching entries from playerList
desktop_path = os.path.expanduser("~/Desktop")
desktop_files = os.listdir(desktop_path)


for file in desktop_files:
    if file.endswith(".txt"):
        file = file[:-4]
    
    try: playerList.remove(file)
    except: pass

# Remove duplicates from playerList
playerList = list(dict.fromkeys(playerList))

# remove "log" and "ItemTooltipExport" from playerList
try: playerList.remove("log")
except: pass
try: playerList.remove("itemTooltipExport")
except: pass

# List of Positions for each record icon x, y (at given GUI scale, resolution (4k), window size) 
_4kRecordIconPositionsX = [1842, 2274, 2274, 1842, 1842, 2274]
_4kRecordIconPositionsY = [1155, 1155, 1209, 1209, 1260, 1260]

_2kRecordIconPositionsX = [921, 1137, 1137, 921, 921, 1137]
_2kRecordIconPositionsY = [int(i) / 2 for i in _4kRecordIconPositionsY]

fastRecordIconPositionsX = [1136, 1171, 1207, 1244, 1280, 1316, 1352, 1388, 1424, 
1424, 
1424, 1388, 1352, 1316, 1280, 1244, 1207, 1171, 1136, 
1136, 1171, 1207, 1244, 1280, 1316, 1352, 1388]

fastRecordIconPositionsY = [611, 611, 611, 611, 611, 611, 611, 611, 611, 
637, 
677, 677, 677, 677, 677, 677, 677, 677, 677,
637, 637, 637, 637, 637, 637, 637, 637]

assert len(fastRecordIconPositionsX) == len(fastRecordIconPositionsY)

# create a list of tuples from the 2k lists
recordIconPositions = list(zip(_2kRecordIconPositionsX, _2kRecordIconPositionsY))

# create a list of tuples from the 4k lists
_4krecordIconPositions = list(zip(_4kRecordIconPositionsX, _4kRecordIconPositionsY))

# create an extended list with values intrapolated in between the given values
recordIconPositionsExtended = []
for i in range(0, 5):
    recordIconPositionsExtended.append(recordIconPositions[i])
    recordIconPositionsExtended.append(((recordIconPositions[i][0] + recordIconPositions[i+1][0]) / 2, (recordIconPositions[i][1] + recordIconPositions[i+1][1]) / 2))

fastIconPositions = list(zip(fastRecordIconPositionsX, fastRecordIconPositionsY))

def scrape(resolution = "2k", fast=False, extended=False, dur=None):
    # Scrape Records for Each Player in playerList
    for i, player in enumerate(playerList):

        if sorted(playerList) == sorted(desktop_files): 
            print("All records have been scraped.")
            break

        with trange(len(playerList), desc="Processing players", position=0) as t:
            t.update(i)

            # Open Chat
            pyautogui.press("t")

            # Assemble Command String
            command = "/rekorde" + " " + player

            # Execute Command
            pyautogui.typewrite(command, 
                                #interval=0.01
                                )
            pyautogui.press("enter")

            # mooth smoothly through all positions x, y, taking x and y from the respective list

            pyautogui.moveTo(x = 500, y = 500, duration=0.1)

            if resolution == "4k": coordTpls = _4krecordIconPositions
            elif resolution == "2k": coordTpls = recordIconPositions
            if extended: coordTpls = recordIconPositionsExtended
            else: coordTpls = recordIconPositions

            if fast: 
                coordTpls = fastIconPositions

            duration = 0.7 if not fast else 0.05

            if dur is not None: duration = float(dur)

            if fast or dur is not None: sleep = duration

            for i in range(0, (len(coordTpls))):
                pyautogui.moveTo(x = coordTpls[i], duration=duration)
                # pyautogui.click()
                pyautogui.sleep(sleep)

            pyautogui.press("e")

            # Wait 300 ms until next player] 
            if fast: pyautogui.sleep(0.1)
            else: pyautogui.sleep(0.3)

            # Rename file on desktop
            new_filename = player + ".txt"
            desktop_path = os.path.expanduser("~/Desktop")
            old_filepath = os.path.join(desktop_path, "ItemTooltipExport.txt")
            new_filepath = os.path.join(desktop_path, new_filename)
            os.rename(old_filepath, new_filepath)

            # if log: print(f"Record for {player} <{i+1}> scraped successfully.\n")

def verifyDesktop():
    # verify the linecount for each file, if any file has more or less than 21 lines, print the filename, else give an OK statement, if there is no file for a player, also point that out. List all players in each "category" at once, not as individual line
    for player in playerList:
        try:
            with open(os.path.join(desktop_path, player + ".txt"), "r") as file:
                try: 
                    lines = file.readlines()
                    if len(lines) != 27:
                        print(player + " has " + str(len(lines)) + " lines.")
                except: pass
        except FileNotFoundError:
            print(player + " has no file.")
    print("All files checked.")

def clear():
    try: os.remove(os.path.join(desktop_path, "ItemTooltipExport.txt"))
    except: pass
    try: os.remove(os.path.join(desktop_path, "log.txt"))
    except: pass

In [6]:
# clear()

pyautogui.sleep(5)

scrape(fast=True, dur=0.02)

pyautogui.sleep(2)

clear()

verifyDesktop()

Processing players:  99%|█████████▉| 97/98 [00:03<00:00, 25.14it/s]


All files checked.
